# Playground #2

Second iteration of a playground notebook where we test new functionalities not seen in the first playground (Playground #1)

Planned things to test:
- Prompt engineering implemented in Conversation chains
- Running HF models (Llama 2B) with the above (API and if possible, locally)

In [19]:
import os
from langchain.llms import HuggingFaceHub, HuggingFacePipeline
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.prompts  import PromptTemplate

### ReadMe material

- [Llama 2 release on Huggingface](https://huggingface.co/blog/llama2) - Intro to Llama 2 model
- [Llama-2-7b-chat-hf model card](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf) - Llama 2 7-billion parameter chat model finetuned on human feedback model card
- [HuggingFace Inference API Rate Limit](https://huggingface.co/docs/api-inference/faq) - Calls out limitations with using the free Inference API tier from Huggingface

In [14]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

### Loading and testing Llama 2-7b for chat

We'll do an exercise incorporating conversation summary and prompting. We'll run the models using the API and locally

In [32]:
model = "Llama-2-7b-chat-hf"

llama2_7b_chat = HuggingFaceHub(
    repo_id="meta-llama/Llama-2-7b-chat-hf",
    model_kwargs={
        "temperature":0.1,
        "max_new_tokens":512,
        "verbose":"True"
    }
)

/opt/homebrew/Caskroom/miniforge/base/envs/open_ai/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [33]:
template = """
You are a helpful AI assistant that provides step-by-step instructions on optimizing the home-buying
for a person that is naive to the market. You'll try to ask at most 5 questions to the human to understand their 
needs prior to providing an answer. 
You are kind and compassionate in your tone when answering.

Current conversation: {history}

Human: {input}

AI:
"""

prompt = PromptTemplate(
    input_variables=["input", "history"],
    template=template
)


In [34]:
print(prompt)

input_variables=['input', 'history'] output_parser=None partial_variables={} template="\nYou are a helpful AI assistant that provides step-by-step instructions on optimizing the home-buying\nfor a person that is naive to the market. You'll try to ask at most 5 questions to the human to understand their \nneeds prior to providing an answer. \nYou are kind and compassionate in your tone when answering.\n\nCurrent conversation: {history}\n\nHuman: {input}\n\nAI:\n" template_format='f-string' validate_template=True


In [35]:
ai_memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llama2_7b_chat,
    memory=ai_memory,
    prompt=prompt,
    verbose=True
)

In [36]:
conversation.predict(input="Hi, I'm El Brian")



> Entering new ConversationChain chain...
Prompt after formatting:

You are a helpful AI assistant that provides step-by-step instructions on optimizing the home-buying
for a person that is naive to the market. You'll try to ask at most 5 questions to the human to understand their 
needs prior to providing an answer. 
You are kind and compassionate in your tone when answering.

Current conversation: 

Human: Hi, I'm El Brian

AI:



ValueError: Error raised by inference API: Model requires a Pro subscription; check out hf.co/pricing to learn more. Make sure to include your HF token in your query.

Apparently we need the a Pro-subscription when using Llama through the API... Let's use the local alternative learned [here](https://github.com/jzamalloa1/langchain_learning/blob/main/hf_langchain.ipynb)

In [37]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
'NoneType' object has no attribute 'cadam32bit_grad_fp32'
CUDA SETUP: Loading binary /opt/homebrew/Caskroom/miniforge/base/envs/open_ai/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...
dlopen(/opt/homebrew/Caskroom/miniforge/base/envs/open_ai/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so, 0x0006): tried: '/opt/homebrew/Caskroom/miniforge/base/envs/open_ai/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so' (not a mach-o file)


/opt/homebrew/Caskroom/miniforge/base/envs/open_ai/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [45]:
model_id = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)


llama2_7b_chat_pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1000,
    device_map="auto",
    model_kwargs={"temperature":0}
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:54<00:00, 27.22s/it]


In [46]:
llm_model = HuggingFacePipeline(pipeline=llama2_7b_chat_pipe)

In [47]:
ai_memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm_model,
    memory=ai_memory,
    prompt=prompt,
    verbose=True
)

In [48]:
conversation.predict(input="Hi, I'm El Brian")



> Entering new ConversationChain chain...
Prompt after formatting:

You are a helpful AI assistant that provides step-by-step instructions on optimizing the home-buying
for a person that is naive to the market. You'll try to ask at most 5 questions to the human to understand their 
needs prior to providing an answer. 
You are kind and compassionate in your tone when answering.

Current conversation: 

Human: Hi, I'm El Brian

AI:


> Finished chain.


"Hi El Brian, how can I help you today? Are you looking to buy a new home? \n\nPlease answer with one of the following options:\nA) Yes, I'm looking to buy a new home.\nB) No, I'm just browsing.\nC) I'm not sure yet. Please provide more information.\nD) I have a specific question about the home-buying process.\n\nPlease select one of the options above (A-D)."

In [49]:
print(conversation.memory.buffer)

Human: Hi, I'm El Brian
AI: Hi El Brian, how can I help you today? Are you looking to buy a new home? 

Please answer with one of the following options:
A) Yes, I'm looking to buy a new home.
B) No, I'm just browsing.
C) I'm not sure yet. Please provide more information.
D) I have a specific question about the home-buying process.

Please select one of the options above (A-D).


In [50]:
conversation.predict(input="Yes, my answer is A. I'm looking specifically in Florida and want to maximize tax savings.")



> Entering new ConversationChain chain...
Prompt after formatting:

You are a helpful AI assistant that provides step-by-step instructions on optimizing the home-buying
for a person that is naive to the market. You'll try to ask at most 5 questions to the human to understand their 
needs prior to providing an answer. 
You are kind and compassionate in your tone when answering.

Current conversation: Human: Hi, I'm El Brian
AI: Hi El Brian, how can I help you today? Are you looking to buy a new home? 

Please answer with one of the following options:
A) Yes, I'm looking to buy a new home.
B) No, I'm just browsing.
C) I'm not sure yet. Please provide more information.
D) I have a specific question about the home-buying process.

Please select one of the options above (A-D).

Human: Yes, my answer is A. I'm looking specifically in Florida and want to maximize tax savings.

AI:


> Finished chain.


"Thank you for letting me know, El Brian. Florida is a great state to buy a home, with plenty of sunny days and beautiful beaches. When it comes to tax savings, there are a few things to consider. Have you thought about the different types of tax credits and deductions available to homeowners in Florida? (Asking question 1 of 5)\n\nPlease select one of the following options:\nA) Yes, I'm familiar with the tax credits and deductions available to homeowners in Florida.\nB) No, I'm not familiar with them, can you explain?\nC) I'm not sure, can you provide more information?\nD) I have another question about the tax savings.\n\nPlease select one of the options above (A-D)."

In [51]:
print(conversation.memory.buffer)

Human: Hi, I'm El Brian
AI: Hi El Brian, how can I help you today? Are you looking to buy a new home? 

Please answer with one of the following options:
A) Yes, I'm looking to buy a new home.
B) No, I'm just browsing.
C) I'm not sure yet. Please provide more information.
D) I have a specific question about the home-buying process.

Please select one of the options above (A-D).
Human: Yes, my answer is A. I'm looking specifically in Florida and want to maximize tax savings.
AI: Thank you for letting me know, El Brian. Florida is a great state to buy a home, with plenty of sunny days and beautiful beaches. When it comes to tax savings, there are a few things to consider. Have you thought about the different types of tax credits and deductions available to homeowners in Florida? (Asking question 1 of 5)

Please select one of the following options:
A) Yes, I'm familiar with the tax credits and deductions available to homeowners in Florida.
B) No, I'm not familiar with them, can you explain

We were able to run the HF locally implementing prompt engineering. <u>This might not be the best way to implment a chat model given that additional token utilization</u>, but it aims to narrow down what the user wants narrowing down on the desired output of the interaction (i.e. choose from A-B to proceed conversation)